# Sem Tech Project Gruppe 1

## Get whole Company List

In [15]:
# pip install sparqlwrapper
# pip install pandas
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

### Die aktuelle Query, welche zu jeder Company auch noch die SubCompanies liefert

query="""SELECT ?company ?besteht_aus WHERE {
  ?company wdt:P31 wd:Q4830453.
  OPTIONAL { ?company wdt:P1830 ?besteht_aus. }
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

results = get_results(endpoint_url, query)

# Länge der Liste
print(len(results["results"]["bindings"]))

for result in results["results"]["bindings"][:10]:
    print(result)
    pass

206154
{'company': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q66'}, 'besteht_aus': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q8793'}}
{'company': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q66'}, 'besteht_aus': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q374451'}}
{'company': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q66'}, 'besteht_aus': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q570930'}}
{'company': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q66'}, 'besteht_aus': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q795324'}}
{'company': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q66'}, 'besteht_aus': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q890212'}}
{'company': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q66'}, 'besteht_aus': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q961568'}}
{'company': {'type': 'uri', 'value': 'htt

## Define Company Object Class

In [16]:
class CompanyObject:
    def __init__(self, englishName, list_Languages, list_Lables, list_Aliases, 
                 TwitterUsername, TwitterID, jurisdiction, language, compID=0):
        
        self.compID=compID
        self.englishName=englishName
        self.list_Languages=list_Languages
        self.list_Lables=list_Lables
        self.list_Aliases=list_Aliases
        self.isSubsidiaryOf=isSubsidiaryOf

# Crawl HTML Code

In [18]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


### First Tests of HTML Extraction

response = requests.get(
	url="https://www.wikidata.org/wiki/Q20716",
)
soup = BeautifulSoup(response.content, 'html.parser')

languages=[]
lables=[]

table = soup.find_all("tr", class_=re.compile("wikibase-entitytermsforlanguageview"))

englishName=table[0].parent.find("div").find("span").contents[0]


for elem in table:
    parentElement=elem.parent
    # Add Language Name to the Array
    languages.append(parentElement.find("th").contents[0])
    # Add Label to the other Array
    lables.append(parentElement.find("div").find("span").contents[0])
    print("I found Language: ",parentElement.find("th").contents[0])

I found Language:  English


# Crawl Json

In [20]:
import requests, json
from bs4 import BeautifulSoup
import re
import pandas as pd

results = get_results(endpoint_url, query)

def crawl(compID):
    response = requests.get(
        url=("https://www.wikidata.org/wiki/Special:EntityData/"+compID+".json"))
    data = json.loads(response.text)
    df = pd.DataFrame(data['entities'][compID]['labels'])
    
    languages=df.iloc[0]
    labels=df.iloc[1]
    
#   df = pd.DataFrame(data['entities'][compID]['aliases'])
    
#   aliases = (df.iloc[0],df.iloc[1])
    
### TODO: Get further Data from HTML
    
    
    
#    compObject = CompanyObject('Test',
#                             languages, labels, aliases
#                             )




crawl('Q1418')
# Länge der Liste
#print(len(results["results"]["bindings"]))

for result in results["results"]["bindings"][:10]:
    pass

ValueError: All arrays must be of the same length

In [83]:
from owlready2 import *

onto_path.append("/Users/leonhardziegler/Library/Mobile Documents/com~apple~CloudDocs/UniBw/Master/SematischeTech/Übungen/git_new")
#onto.save(file = "filename or fileobj", format = "rdfxml")
onto = get_ontology("file://names_onto.owl").load()
#default_world.set_backend(filename = "names_onto.owl")


In [68]:
###test 

#company_instance = onto.owl:Company("Q1418"))
"""with onto:
    class Language(Thing):
        namespace = onto
        pass

with onto:
    class Dialect(Language):
        namespace = onto
        pass
"""

language_instance = Language("French")

dialect_instance = Language("Bavarian")

print(list(onto.individuals()))

print(language_instance.name)




None
[webprotege.stanford.edu.R1tTpXLvo8OlfGKD1B9sD8, webprotege.stanford.edu.R8ov2vaR2o6xwusUMLP6Ttc, webprotege.stanford.edu.R9Mm4K8OsE35OMG2l8lzmMB, names_onto.English, names_onto.French, names_onto.Bavarian]
French


In [84]:
onto.save()


In [ ]:
### Example Instantiates

# Example Instance of Mother Company
NokiaMother = CompanyObject('Nokia',['Englisch','Japan'],['Nokia','JapanNokia'],'AG','More than 1000',
                     'More than 1 Billion','Nok_CEO','Nok_CIO', 'null')

# How to Access it
print("Name: ",NokiaMother.englishName,"\n")

# Example Instance of Daughter Company
NokiaDaughter = CompanyObject('Nokia_Daughter',['Englisch','Japan'],['NokiaDaugh','JapanNokiaDaugh'],'AG','More than 10',
                               'More than 1 Million','Nok_CEODaugh','Nok_CIODaugh', NokiaMother)

# How to Access it # Access to MotherCompany( if available)
print("Name: ",NokiaDaughter.englishName," is a subsidiary of: ",NokiaDaughter.isSubsidiaryOf.englishName)